In [19]:
from openai import OpenAI
import IPython
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
serpapi_api_key = os.getenv("SERPAPI_API_KEY")

client = OpenAI()

In [2]:
def get_completion(messages,  model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

## 1. Few-shot In-context Learning

### With

In [3]:
prompt = """Your task is to classify an input text (delimited by ```) as either offensive or non-offensive.

Use the following examples to help with steering your responses:

Text: I love you
Output: non-offensive

Text: I dislike all those people working at the company
Output: offensive

Text: I think this feature is not ideal
Output: non-offensive

Text: Those people are so stupid
Output: offensive

Text: {user_input}
Output:
"""

message = [
    {
        "role": "user",
        "content": prompt.format(user_input="```I respectfully disagree with your opinion.```")
    }
]


response = get_completion(message)
print(response)

non-offensive


### Without

In [4]:
prompt = """Your task is to classify an input text (delimited by ```) as either offensive or non-offensive.
"""

message = [
    {
        "role": "user",
        "content": prompt.format(user_input="```I respectfully disagree with your opinion.```")
    }
]


response = get_completion(message)
print(response)

To classify an input text as either offensive or non-offensive, you can use a machine learning model trained on a labeled dataset. The model should be able to predict the class of a given text based on its features.

Here is a high-level approach to accomplish this task:

1. Data Collection: Gather a dataset of labeled offensive and non-offensive texts. You can manually label the data or use existing labeled datasets available online.

2. Data Preprocessing: Clean and preprocess the text data by removing any irrelevant information, special characters, or stopwords. Convert the text into a numerical representation suitable for machine learning algorithms, such as word embeddings or TF-IDF vectors.

3. Model Training: Train a machine learning model on the preprocessed data. There are various algorithms you can use, such as logistic regression, support vector machines, or deep learning models like recurrent neural networks (RNNs) or transformers.

4. Model Evaluation: Evaluate the trained

## 2. Chain of Thought (CoT)
Movie Recommendation Asisstant using CoT

In [8]:
# define list of movies

movies = """
The Enigma Code
Category: Historical Drama
Rating: 8.3/10
Description: Set during World War II, this gripping historical drama follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. His efforts contribute significantly to the Allies' victory.
Actors: Benedict Cumberbatch, Keira Knightley, Matthew Goode
Language: English
Release Date: March 15, 2014
Award Winner: Academy Award for Best Adapted Screenplay

Shadows of the Samurai
Category: Action/Adventure
Rating: 7.9/10
Description: In feudal Japan, a skilled samurai seeks vengeance against the corrupt warlord who murdered his master. With his swordsmanship and determination, he embarks on a dangerous journey to restore justice.
Actors: Ken Watanabe, Tadanobu Asano, Rinko Kikuchi
Language: Japanese
Release Date: November 7, 2017
Award Winner: None

Mind Games
Category: Psychological Thriller
Rating: 8.1/10
Description: A renowned psychologist becomes entangled in a twisted game of cat and mouse with a patient who harbors dark secrets. As their sessions progress, the lines between reality and deception blur, leading to a mind-bending climax.
Actors: Leonardo DiCaprio, Natalie Portman, Michael Fassbender
Language: English
Release Date: August 22, 2019
Award Winner: None

La Casa del Tango
Category: Musical/Drama
Rating: 8.7/10
Description: In the vibrant world of Buenos Aires, a passionate tango dancer finds love and inspiration amidst the backdrop of political unrest. This musical drama explores the power of dance and the pursuit of dreams.
Actors: Antonio Banderas, Penélope Cruz, Javier Bardem
Language: Spanish
Release Date: June 5, 2020
Award Winner: Golden Globe for Best Foreign Language Film

Timeless Love
Category: Romance/Fantasy
Rating: 7.5/10
Description: A magical encounter transports a modern-day writer back in time to Victorian England, where she falls in love with a charming aristocrat. As they navigate the complexities of time, their love is put to the ultimate test.
Actors: Rachel McAdams, Tom Hiddleston, Emma Thompson
Language: English
Release Date: February 14, 2022
Award Winner: None

The Pursuit of Justice
Category: Legal Drama
Rating: 8.4/10
Description: Inspired by true events, this gripping legal drama follows a determined lawyer's fight against a powerful pharmaceutical company responsible for a life-threatening drug. The courtroom battle becomes a quest for justice and redemption.
Actors: Denzel Washington, Viola Davis, Michael B. Jordan
Language: English
Release Date: October 10, 2022
Award Winner: None

The Forgotten Island
Category: Adventure/Mystery
Rating: 7.6/10
Description: A group of explorers stumbles upon a mysterious island believed to be uninhabited. As they uncover the island's secrets, they encounter deadly challenges and unravel an ancient civilization's enigma.
Actors: Chris Pratt, Bryce Dallas Howard, Tom Holland
Language: English
Release Date: July 2, 2023
Award Winner: None

The Silent Witness
Category: Crime/Thriller
Rating: 8.2/10
Description: A talented forensic pathologist becomes entangled in a high-stakes murder investigation when she discovers crucial evidence that points to a powerful criminal network. With her life on the line, she must outsmart the perpetrators.
Actors: Emily Blunt, Jake Gyllenhaal, Mark Ruffalo
Language: English
Release Date: November 18, 2023
Award Winner: None

A Tale of Two Worlds
Category: Fantasy/Adventure
Rating: 7.8/10
Description: When a young orphan discovers a magical portal to a parallel universe, she embarks on a thrilling adventure to save both realms from an impending disaster. Along the way, she learns about the power of friendship and self-belief.
Actors: Millie Bobby Brown, Tom Holland, Helena Bonham Carter
Language: English
Release Date: April 5, 2024
Award Winner: None

A Symphony of Souls
Category: Music/Drama
Rating: 9.0/10
Description: Set against the backdrop of a renowned symphony orchestra, this emotionally charged drama explores the lives and intertwining stories of its members. Through the power of music, they find solace, love, and redemption.
Actors: Meryl Streep, Tom Hanks, Cate Blanchett
Language: English
Release Date: December 25, 2024
Award Winner: None
"""

Then, define the steps need the model should performs.

After that, we ask the model to provide the reasoning steps and the final answer.

### With

In [17]:
system_message = """
Your task is to make movie recommendation based on a user request (delimited by ```).

Step 1: Check if the user is asking about movies. If the user is not asking about movies, just respond "Please ask something about movies!".

Step 2: If the user is asking for a movie recommendation, check if they have any specific requests or interests.

Step 3: Check if there are any movie/s we can recommend from the following: {movies}

Step 4: Prepare a response to the user with the movie recommendation/s. The recommendation have to be about movies that are available in the list above. The response needs to have a friendly and helpful tone.

Return a response with the following reasoning steps and final answer to the user:
Step 1: <Step 1 reasoning>
Step 2: <Step 2 reasoning>
Step 3: <Step 3 reasoning>
Step 4: <Step 4 reasoning>
"""


message = [
    {
        "role": "system",
        "content": system_message.format(movies=movies)
    },
    {
        "role": "user",
        "content": "```Do you have any drama movies?```"
    }
]

movie_recommendation_response = get_completion(message, temperature=0, max_tokens=500)
print(movie_recommendation_response)

Step 1: The user is asking about movies.

Step 2: The user is specifically asking for drama movies.

Step 3: We have several drama movies available in our list, including "The Enigma Code", "La Casa del Tango", "The Pursuit of Justice", and "A Symphony of Souls".

Step 4: Based on your request for drama movies, I would recommend the following options:

1. "The Enigma Code": This historical drama set during World War II follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. It has a rating of 8.3/10 and features actors like Benedict Cumberbatch and Keira Knightley.

2. "La Casa del Tango": This musical drama is set in the vibrant world of Buenos Aires and explores the power of dance and the pursuit of dreams. It has a rating of 8.7/10 and features actors like Antonio Banderas and Penélope Cruz.

3. "The Pursuit of Justice": This gripping legal drama is inspired by true events and follows a determined lawyer's fight against a po

### Without

In [16]:
system_message = """
Your task is to make movie recommendation based on a user request (delimited by ```).
"""


message = [
    {
        "role": "system",
        "content": system_message.format(movies=movies)
    },
    {
        "role": "user",
        "content": "```Do you have any drama movies?```"
    }
]

movie_recommendation_response = get_completion(message, temperature=0, max_tokens=500)
print(movie_recommendation_response)

Yes, I have some drama movie recommendations for you. Here are a few:

1. "The Shawshank Redemption" - This classic drama film tells the story of a banker who is sentenced to life in Shawshank State Penitentiary and forms a bond with a fellow inmate.

2. "The Godfather" - This iconic crime drama follows the Corleone family and their involvement in organized crime in New York City.

3. "Schindler's List" - Based on a true story, this powerful drama depicts the efforts of Oskar Schindler to save Jews during the Holocaust.

4. "The Pursuit of Happyness" - Starring Will Smith, this inspiring drama tells the story of a struggling salesman who becomes homeless with his young son and eventually overcomes adversity.

5. "The Fault in Our Stars" - This heartwarming drama is based on the bestselling novel and follows the love story of two teenagers who meet at a cancer support group.

I hope you find these recommendations enjoyable!


## 3. Prompt Chaining
How separate prompts to achieve a specific task.

It different with CoT, where it list all the reasoning steps.

While in `Prompt Chaining`, we ask the model to extract only the final response to the user.

In [18]:
# Prompt 1: step by step reasoning (provided above)
# prompt 2: extract only the final response to the user


system_message_2 = """
You will be given a list of steps that a model has responded with. Your task is to extract only the full response in Step 4 from the following text: {movie_recommendation_response}

Step 4:
"""

messages = [
    {
        "role": "system",
        "content": system_message_2.format(movie_recommendation_response=movie_recommendation_response)
    }
]

final_response = get_completion(messages, temperature=0)

print(final_response)

Based on your request for drama movies, I would recommend the following options:

1. "The Enigma Code": This historical drama set during World War II follows the life of Alan Turing, a brilliant mathematician tasked with cracking the Enigma code used by the Nazis. It has a rating of 8.3/10 and features actors like Benedict Cumberbatch and Keira Knightley.

2. "La Casa del Tango": This musical drama is set in the vibrant world of Buenos Aires and explores the power of dance and the pursuit of dreams. It has a rating of 8.7/10 and features actors like Antonio Banderas and Penélope Cruz.

3. "The Pursuit of Justice": This gripping legal drama is inspired by true events and follows a determined lawyer's fight against a powerful pharmaceutical company. It has a rating of 8.4/10 and features actors like Denzel Washington and Viola Davis.

4. "A Symphony of Souls": This emotionally charged drama is set against the backdrop of a renowned symphony orchestra and explores the lives and intertwini

## 4. ReAct
Combining LLMs and external tools to achieve a task.

In this example, use LLMs and SerpAPI (search engine)

In [26]:
from langchain.agents import load_tools, AgentType, initialize_agent
from langchain.llms import OpenAI

# model
llm = OpenAI(temperature=0)

# tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [28]:
agent.run("Which team won the 2023 NBA finals?")



> Entering new AgentExecutor chain...
 I need to find out what happened in the 2023 NBA finals
Action: Search
Action Input: 2023 NBA finals
Observation: {'title': 'NBA', 'rankings': 'NBA Finals', 'thumbnail': 'https://serpapi.com/searches/65795576234884d4645f5550/images/40f2d356f4383e465924765afd832f852120e587ad9cd68288fc7c82b19c6e96.png'}
Thought: I need to find out which team won
Action: Search
Action Input: 2023 NBA finals winner
Observation: {'title': 'NBA', 'rankings': 'NBA Finals', 'thumbnail': 'https://serpapi.com/searches/6579557a2bc7d8d86dc5dab7/images/3643ed19624701cc91b778a9614da8493f510090fb4f166ac8b9656d16bb4fa1.png'}
Thought: I now know the final answer
Final Answer: The Los Angeles Lakers won the 2023 NBA finals.

> Finished chain.


'The Los Angeles Lakers won the 2023 NBA finals.'

## 5. Retrieval Augmentation Generation (RAG)
Apply RAG

In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

import urllib

In [38]:
# load the data from github
f = urllib.request.urlopen("https://raw.githubusercontent.com/comet-ml/comet-llmops/main/data/kar-gpt.txt")
text_data = f.read().decode("utf-8")

# split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator=" ")
texts = text_splitter.split_text(text_data)

# embeddings obtained from OpenAI
embeddings = OpenAIEmbeddings()

In [39]:
# test search
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

query = "What is the course about?"
docs = docsearch.similarity_search(query)

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' This course is about understanding and appreciating how chat GPT works, and how to develop a transformer neural network. It requires proficiency in Python and some basic understanding of calculus and statistics.\nSOURCES: 1, 7, 107, 108'}

### Reference:
chain_type in Question-Answering: 
- https://towardsdatascience.com/
- 4-ways-of-question-answering-in-langchain-188c6707cc5a
- https://medium.com/@kbdhunga/enhancing-conversational-ai-the-power-of-langchains-question-answer-framework-4974e1cab3cf 
- https://www.youtube.com/watch?v=Xn85BkrVAyk
- https://www.youtube.com/watch?v=OTL4CvDFlro
- https://www.youtube.com/watch?v=DXmiJKrQIvg&t=1s

1. Chain type = stuff
2. Chain type = map_reduce
3. Chain type = refine
4. Chain type = map_rerank